In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


# balancing dataset

In [3]:
import os
sc_path = 'C:\\_TBZ mindenes\\sewerbot_ai\\src\\pictures\\data'
["{}: {}".format(root, len(files)) for root, dirs, files in os.walk(sc_path)]


['C:\\_TBZ mindenes\\sewerbot_ai\\src\\pictures\\data: 0',
 'C:\\_TBZ mindenes\\sewerbot_ai\\src\\pictures\\data\\end: 8698',
 'C:\\_TBZ mindenes\\sewerbot_ai\\src\\pictures\\data\\notEnd: 14118']

In [4]:
min_len = min([len(files) for root, dirs, files in os.walk(sc_path)][1:])
min_len

8698

In [5]:
import random
for root, dirs, files in os.walk(sc_path):
    if (len(files) == 0):
        continue
    del_count = len(files)-min_len
    random.shuffle(files)
    for f in files[:del_count]:
        os.remove(os.path.join(root, f))

In [6]:
["{}: {}".format(root, len(files)) for root, dirs, files in os.walk(sc_path)]

['C:\\_TBZ mindenes\\sewerbot_ai\\src\\pictures\\data: 0',
 'C:\\_TBZ mindenes\\sewerbot_ai\\src\\pictures\\data\\end: 8698',
 'C:\\_TBZ mindenes\\sewerbot_ai\\src\\pictures\\data\\notEnd: 8698']

# train

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
data_directory = 'C:\\_TBZ mindenes\\sewerbot_ai\\src\\pictures\\data'
dataset = ImageFolder(data_directory, transform=transform)

In [29]:
batch_size = 144
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [30]:
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)

Using cache found in C:\Users\user/.cache\torch\hub\pytorch_vision_main


In [31]:
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),  
    nn.Linear(256, 2) # binary classification
)


In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [33]:
from tqdm import tqdm
import numpy as np

In [34]:
num_epochs = 20
checkpoint_interval = 10 
model_name = 'pretrain_3lenghts'

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model.to(DEVICE)
criterion.to(DEVICE)

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(data_loader, desc=f'Epoch {epoch + 1}'):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(data_loader)
    epoch_accuracy = 100 * correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

        # Save a checkpoint every checkpoint_interval epochs
    if (epoch + 1) % checkpoint_interval == 0:
        checkpoint_name = f'{model_name}_checkpoint_{epoch + 1}.pth'
        # checkpoint_path = os.path.join(checkpoint_dir, checkpoint_name)
        torch.save(model.state_dict(), checkpoint_name)
        print(f'Checkpoint saved: {checkpoint_name}')

Epoch 1:   0%|          | 0/121 [00:00<?, ?it/s]

Epoch 1: 100%|██████████| 121/121 [04:02<00:00,  2.00s/it]


Epoch 1/20, Loss: 0.0609, Accuracy: 97.52%


Epoch 2: 100%|██████████| 121/121 [03:01<00:00,  1.50s/it]


Epoch 2/20, Loss: 0.0289, Accuracy: 98.84%


...

Epoch 9/20, Loss: 0.0099, Accuracy: 99.60%


Epoch 10: 100%|██████████| 121/121 [02:54<00:00,  1.44s/it]


Epoch 10/20, Loss: 0.0099, Accuracy: 99.63%
Checkpoint saved: pretrain_3lenghts_checkpoint_10.pth


Epoch 11:   8%|▊         | 10/121 [00:15<02:56,  1.59s/it]


KeyboardInterrupt: 

In [11]:
torch.save(model.state_dict(), 'test_model.pth')

In [13]:
# Load the saved model
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)  # Load the same architecture
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 2)
)
model.load_state_dict(torch.load('test_model.pth'))
model.eval()  # Set the model to evaluation mode


Using cache found in C:\Users\user/.cache\torch\hub\pytorch_vision_main


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
import cv2
from PIL import Image


In [26]:
cap = cv2.VideoCapture("C:\\_TBZ mindenes\\sewerbot_ai\\src\\videos\\test_2.avi")
# cap = cv2.VideoCapture("http://192.168.1.1:8081/")
class_labels = ['end', 'notEnd']

while True:
    ret, frame_og = cap.read()

    if not ret:
        break

    # Preprocess the frame
    frame = cv2.cvtColor(frame_og, cv2.COLOR_BGR2RGB)  # Convert to RGB
    frame = cv2.resize(frame, (224, 224))  # Resize to match your model's input size
    frame = transform(Image.fromarray(frame))  # Apply the same preprocessing transform as in training

    # Make a prediction
    with torch.no_grad():
        output = model(frame.unsqueeze(0))

    _, predicted_class = torch.max(output, 1)
    predicted_label = class_labels[predicted_class.item()]

    # Display the frame with the prediction
    frame = cv2.cvtColor(frame.permute(1, 2, 0).numpy(), cv2.COLOR_RGB2BGR)  # Convert back to BGR
    cv2.putText(frame_og, f"Prediction: {predicted_label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("IP Camera Stream", frame_og)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 